<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [2]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

**Read the data**

In [5]:
PATH_TO_DATA = Path('flight-delays-fall-2018/')

In [6]:
train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')

In [7]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [8]:
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')

In [9]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


**Create only one feature - “flight” (this you need to improve - add more features)**

In [10]:
train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

In [25]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


In [26]:
full_df = pd.concat((train_df, test_df))

/Users/Svetlana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [31]:
# Concatenate for preprocessing
train_split = train_df.shape[0]

In [27]:
# Hour and minute
full_df['hour'] = full_df['DepTime'] // 100
full_df.loc[full_df['hour'] == 24, 'hour'] = 0
full_df.loc[full_df['hour'] == 25, 'hour'] = 1
full_df['minute'] = full_df['DepTime'] % 100

# Season
full_df['summer'] = (full_df['Month'].isin(['c-6', 'c-7', 'c-8'])).astype(np.int32)
full_df['autumn'] = (full_df['Month'].isin(['c-9', 'c-10', 'c-11'])).astype(np.int32)
full_df['winter'] = (full_df['Month'].isin(['c-12', 'c-1', 'c-2'])).astype(np.int32)
full_df['spring'] = (full_df['Month'].isin(['c-3', 'c-4', 'c-5'])).astype(np.int32)

# Daytime
full_df['daytime'] = pd.cut(full_df['hour'], bins=[0, 6, 12, 18, 23], include_lowest=True)

In [28]:
full_df.head()

,DayOfWeek,DayofMonth,DepTime,Dest,Distance,Month,Origin,UniqueCarrier,dep_delayed_15min,flight,hour,minute,summer,autumn,winter,spring,daytime
0,c-7,c-21,1934,DFW,732,c-8,ATL,AA,N,ATL-->DFW,19,34,1,0,0,0,"(18.0, 23.0]"
1,c-3,c-20,1548,MCO,834,c-4,PIT,US,N,PIT-->MCO,15,48,0,0,0,1,"(12.0, 18.0]"
2,c-5,c-2,1422,CLE,416,c-9,RDU,XE,N,RDU-->CLE,14,22,0,1,0,0,"(12.0, 18.0]"
3,c-6,c-25,1015,MEM,872,c-11,DEN,OO,N,DEN-->MEM,10,15,0,1,0,0,"(6.0, 12.0]"
4,c-6,c-7,1828,OMA,423,c-10,MDW,WN,Y,MDW-->OMA,18,28,0,1,0,0,"(12.0, 18.0]"


In [29]:
# String to numerical
for col in ['Month', 'DayofMonth', 'DayOfWeek']:
    full_df[col] = full_df[col].apply(lambda x: x.split('-')[1]).astype(np.int32) - 1

# Label Encoding
for col in ['Origin', 'Dest', 'UniqueCarrier', 'daytime']:
    full_df[col] = pd.factorize(full_df[col])[0]

In [46]:
# Categorical columns
cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'Origin', 'Dest', 'UniqueCarrier', 'hour', 'summer', 'autumn', 'winter', 'spring', 'daytime']

# Converting categorical columns to type 'category' as required by LGBM
for c in cat_cols:
    full_df[c] = full_df[c].astype('object')

In [50]:
full_df.head()

,DayOfWeek,DayofMonth,DepTime,Dest,Distance,Month,Origin,UniqueCarrier,dep_delayed_15min,flight,hour,minute,summer,autumn,winter,spring,daytime
0,6,20,1934,0,732,7,0,0,N,ATL-->DFW,19,34,1,0,0,0,0
1,2,19,1548,1,834,3,1,1,N,PIT-->MCO,15,48,0,0,0,1,1
2,4,1,1422,2,416,8,2,2,N,RDU-->CLE,14,22,0,1,0,0,1
3,5,24,1015,3,872,10,3,3,N,DEN-->MEM,10,15,0,1,0,0,2
4,5,6,1828,4,423,9,4,4,Y,MDW-->OMA,18,28,0,1,0,0,1


In [51]:
# Split into train and test
train_df, test_df = full_df.iloc[:train_split], full_df.iloc[train_split:]
train_df.shape, test_df.shape

((100000, 17), (100000, 17))

In [52]:
train_df.dtypes

DayOfWeek            object
DayofMonth           object
DepTime               int64
Dest                 object
Distance              int64
Month                object
Origin               object
UniqueCarrier        object
dep_delayed_15min    object
flight               object
hour                 object
minute                int64
summer               object
autumn               object
winter               object
spring               object
daytime              object
dtype: object

**Remember indexes of categorical features (to be passed to CatBoost)**

In [53]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

array([ 0,  1,  3,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15])

**Allocate a hold-out set (a.k.a. a validation set) to validate the model**

In [54]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

In [55]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

**Train Catboost with default arguments, passing only the indexes of categorical features.**

In [56]:
ctb = CatBoostClassifier(random_seed=17, silent=True)

In [57]:
%%time
ctb.fit(X_train_part, y_train_part,
        cat_features=categ_feat_idx);

CPU times: user 6min 56s, sys: 58.3 s, total: 7min 55s
Wall time: 2min 52s


In [58]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

In [64]:
X_valid[:10]

array([[3, 19, 702, 29, 951, 9, 6, 1, 'PBI-->PHL', 7, 2, 0, 1, 0, 0, 2],
       [1, 13, 707, 0, 224, 5, 51, 0, 'IAH-->DFW', 7, 7, 1, 0, 0, 0, 2],
       [1, 1, 1600, 20, 337, 7, 35, 4, 'OAK-->LAX', 16, 0, 1, 0, 0, 0, 1],
       [3, 20, 2132, 30, 589, 11, 23, 9, 'ORD-->IAD', 21, 32, 0, 0, 1, 0,
        0],
       [1, 8, 1320, 159, 631, 4, 23, 14, 'ORD-->AVP', 13, 20, 0, 0, 0, 1,
        1],
       [2, 13, 1930, 31, 197, 5, 8, 4, 'ONT-->LAS', 19, 30, 1, 0, 0, 0,
        0],
       [6, 14, 1327, 12, 140, 0, 50, 11, 'AUS-->IAH', 13, 27, 0, 0, 1, 0,
        1],
       [3, 26, 1847, 6, 268, 9, 38, 7, 'GRR-->CVG', 18, 47, 0, 1, 0, 0,
        1],
       [2, 18, 1544, 31, 414, 0, 28, 18, 'SFO-->LAS', 15, 44, 0, 0, 1, 0,
        1],
       [4, 20, 730, 0, 190, 0, 96, 12, 'SHV-->DFW', 7, 30, 0, 0, 1, 0, 2]],
      dtype=object)

**We got some 0.756 ROC AUC on the hold-out set.**

In [59]:
roc_auc_score(y_valid, ctb_valid_pred)

0.7991141449947238

**Train on the whole train set, make prediction on the test set. We got ~0.734 in the competition - "Catboost starter" baseline**

In [60]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx);

CPU times: user 9min 27s, sys: 1min 9s, total: 10min 37s
Wall time: 3min 10s


In [63]:
X_test[:10]

array([[2, 24, 615, 17, 598, 6, 139, 14, nan, 'MRY-->PHX', 6, 15, 1, 0,
        0, 0, 3],
       [1, 16, 739, 91, 1235, 3, 11, 4, nan, 'LAS-->HOU', 7, 39, 0, 0, 0,
        1, 2],
       [6, 1, 651, 11, 577, 11, 29, 10, nan, 'GSP-->ORD', 6, 51, 0, 0, 1,
        0, 3],
       [6, 24, 1614, 138, 377, 2, 42, 4, nan, 'BWI-->MHT', 16, 14, 0, 0,
        0, 1, 1],
       [2, 5, 1505, 8, 258, 5, 23, 9, nan, 'ORD-->STL', 15, 5, 1, 0, 0,
        0, 1],
       [3, 21, 854, 27, 529, 1, 15, 5, nan, 'ORF-->DTW', 8, 54, 0, 0, 1,
        0, 2],
       [0, 29, 723, 179, 2277, 3, 19, 19, nan, 'IAD-->LGB', 7, 23, 0, 0,
        0, 1, 2],
       [1, 5, 1544, 1, 1310, 1, 7, 5, nan, 'MSP-->MCO', 15, 44, 0, 0, 1,
        0, 1],
       [2, 10, 1139, 57, 994, 6, 1, 1, nan, 'PIT-->FLL', 11, 39, 1, 0, 0,
        0, 2],
       [2, 10, 1721, 4, 583, 6, 12, 0, nan, 'DFW-->OMA', 17, 21, 1, 0, 0,
        0, 1]], dtype=object)

In [67]:
X_test = test_df.drop(columns=['dep_delayed_15min']).values
X_test[:10]

array([[2, 24, 615, 17, 598, 6, 139, 14, 'MRY-->PHX', 6, 15, 1, 0, 0, 0,
        3],
       [1, 16, 739, 91, 1235, 3, 11, 4, 'LAS-->HOU', 7, 39, 0, 0, 0, 1,
        2],
       [6, 1, 651, 11, 577, 11, 29, 10, 'GSP-->ORD', 6, 51, 0, 0, 1, 0,
        3],
       [6, 24, 1614, 138, 377, 2, 42, 4, 'BWI-->MHT', 16, 14, 0, 0, 0, 1,
        1],
       [2, 5, 1505, 8, 258, 5, 23, 9, 'ORD-->STL', 15, 5, 1, 0, 0, 0, 1],
       [3, 21, 854, 27, 529, 1, 15, 5, 'ORF-->DTW', 8, 54, 0, 0, 1, 0, 2],
       [0, 29, 723, 179, 2277, 3, 19, 19, 'IAD-->LGB', 7, 23, 0, 0, 0, 1,
        2],
       [1, 5, 1544, 1, 1310, 1, 7, 5, 'MSP-->MCO', 15, 44, 0, 0, 1, 0, 1],
       [2, 10, 1139, 57, 994, 6, 1, 1, 'PIT-->FLL', 11, 39, 1, 0, 0, 0,
        2],
       [2, 10, 1721, 4, 583, 6, 12, 0, 'DFW-->OMA', 17, 21, 1, 0, 0, 0,
        1]], dtype=object)

In [68]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [69]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sample_sub.to_csv('ctb_pred.csv')

In [70]:
!head ctb_pred.csv

id,dep_delayed_15min
0,0.017464426522839808
1,0.03604202907279981
2,0.07320794981505266
3,0.44987152047509354
4,0.3216672256507056
5,0.04096050171810034
6,0.18042483280138433
7,0.4957953534516384
8,0.27839693257765397


Now's your turn! Go and improve the model to beat **"A2 baseline (10 credits)"** - **0.75914** LB score. It's crucial to come up with some good features. 

For discussions, stick to the **#a2_kaggle_fall2019** thread in the **mlcourse_ai_news** [ODS Slack](http://opendatascience.slack.com) channel. Serhii Romanenko (@serhii_romanenko) will be there to help. 

Welcome to Kaggle!

<img src='https://habrastorage.org/webt/fs/42/ms/fs42ms0r7qsoj-da4x7yfntwrbq.jpeg' width=50%>
*from the ["Nerd Laughing Loud"](https://www.kaggle.com/general/76963) thread.*